In [13]:
%%writefile variancia.cu
#include <iostream>
#include <fstream>
#include <sstream>
#include <string>
#include <vector>
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/reduce.h>
#include <thrust/extrema.h>
#include <thrust/transform.h>
#include <thrust/iterator/constant_iterator.h>

void read_dual_stock_data(const std::string& file_name, thrust::host_vector<float>& aapl, thrust::host_vector<float>& msft) {
    std::ifstream file(file_name);
    std::string line;

    while (std::getline(file, line)) {
        std::stringstream ss(line);
        std::string apple_str, microsoft_str;

        if (std::getline(ss, apple_str, ',') && std::getline(ss, microsoft_str)) {
            float apple_price = std::stof(apple_str);
            float microsoft_price = std::stof(microsoft_str);
            aapl.push_back(apple_price);
            msft.push_back(microsoft_price);
        }
    }
}

int main() {
    thrust::host_vector<float> h_aapl_prices, h_msft_prices;
    read_dual_stock_data("stocks.txt", h_aapl_prices, h_msft_prices);

    // Transfer data to device
    thrust::device_vector<float> d_aapl_prices = h_aapl_prices;
    thrust::device_vector<float> d_msft_prices = h_msft_prices;

    // Calculate differences between AAPL and MSFT prices
    thrust::device_vector<float> d_diff(d_aapl_prices.size());
    thrust::transform(d_aapl_prices.begin(), d_aapl_prices.end(), d_msft_prices.begin(), d_diff.begin(), thrust::minus<float>());

    // Calculate the mean of differences
    float total_diff = thrust::reduce(d_diff.begin(), d_diff.end(), 0.0f);
    float average_diff = total_diff / d_diff.size();
    std::cout << "Diferença média entre os preços da Apple e Microsoft: " << average_diff << std::endl;

    // Calculate the variance
    thrust::device_vector<float> d_variance(d_diff.size());
    thrust::transform(d_diff.begin(), d_diff.end(),
                      thrust::constant_iterator<float>(average_diff),
                      d_variance.begin(),
                      thrust::minus<float>());  // (xi - μ)

    thrust::transform(d_variance.begin(), d_variance.end(), d_variance.begin(),
                      thrust::square<float>());  // (xi - μ)²

    float variance_sum = thrust::reduce(d_variance.begin(), d_variance.end(), 0.0f);
    float variance = variance_sum / d_diff.size();
    std::cout << "Variância das diferenças: " << variance << std::endl;

    return 0;
}

Overwriting variancia.cu


In [14]:
!nvcc -arch=sm_75 -std=c++14 variancia.cu -o variancia

nvcc fatal   : Cannot find compiler 'cl.exe' in PATH


In [15]:
!./variancia

'.' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


In [16]:
%%writefile variacaodiaria.cu
#include <iostream>
#include <fstream>
#include <sstream>
#include <string>
#include <vector>
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/transform.h>
#include <thrust/iterator/counting_iterator.h>

// Função para ler os preços de ações do arquivo
void read_stock_data(const std::string& file_name, thrust::host_vector<float>& stocks) {
    std::ifstream file(file_name);
    std::string line;

    while (std::getline(file, line)) {
        float price = std::stof(line); // Converte a string para float
        stocks.push_back(price);       // Armazena no vetor
    }
}

// Functor para calcular a diferença entre dias consecutivos
struct daily_gain_functor {
    __host__ __device__
    float operator()(const float& current, const float& previous) const {
        return current - previous;  // Calcula ganho diário
    }
};

int main() {
    thrust::host_vector<float> h_stocks;
    read_stock_data("google.txt", h_stocks);

    size_t total_size = h_stocks.size();
    const size_t block_size = 100000; // Processar 100.000 elementos por vez (ajustável)

    for (size_t i = 0; i < total_size; i += block_size) {
        // Define o tamanho do bloco atual
        size_t current_block_size = std::min(block_size, total_size - i);

        // Vetores no host para o bloco atual
        thrust::host_vector<float> h_stocks_block(h_stocks.begin() + i, h_stocks.begin() + i + current_block_size);

        // Transferir o bloco para a GPU
        thrust::device_vector<float> d_stocks = h_stocks_block;

        if (d_stocks.size() > 1) {
            thrust::device_vector<float> d_daily_gain(d_stocks.size() - 1);
            thrust::transform(d_stocks.begin() + 1, d_stocks.end(), d_stocks.begin(), d_daily_gain.begin(), daily_gain_functor());

            thrust::host_vector<float> h_daily_gain = d_daily_gain;
            for (size_t j = 0; j < h_daily_gain.size(); ++j) {
                std::cout << h_daily_gain[j] << std::endl;
            }
        }
    }

    return 0;
}

Overwriting variacaodiaria.cu


In [17]:
!nvcc -arch=sm_75 -std=c++14 variacaodiaria.cu -o variacaodiaria

nvcc fatal   : Cannot find compiler 'cl.exe' in PATH


In [18]:
!./variacaodiaria

'.' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


In [19]:
%%writefile contasubida.cu
#include <iostream>
#include <fstream>
#include <sstream>
#include <string>
#include <vector>
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/transform.h>
#include <thrust/functional.h>
#include <thrust/count.h>

// Função para ler os preços de ações do arquivo
void read_stock_data(const std::string& file_name, thrust::host_vector<float>& stocks) {
    std::ifstream file(file_name);
    std::string line;

    while (std::getline(file, line)) {
        float price = std::stof(line); // Converte a string para float
        stocks.push_back(price);       // Armazena no vetor
    }
}

// Functor para calcular a diferença entre dias consecutivos
struct daily_gain_functor {
    __host__ __device__
    float operator()(const float& current, const float& previous) const {
        return current - previous;  // Calcula ganho diário
    }
};

// Functor para identificar dias de aumento no preço (diferença positiva)
struct is_positive_gain {
    __host__ __device__
    bool operator()(const float& x) const {
        return x > 0.0f;  // Retorna verdadeiro se o ganho for positivo
    }
};

int main() {
    thrust::host_vector<float> h_stocks;
    read_stock_data("google.txt", h_stocks);

    size_t total_size = h_stocks.size();
    const size_t block_size = 100000; // Tamanho do bloco a ser processado (ajustável)

    int total_positive_gains = 0;

    for (size_t i = 0; i < total_size - 1; i += block_size) {
        size_t current_block_size = std::min(block_size, total_size - i - 1);

        // Bloco atual de dados
        thrust::host_vector<float> h_stocks_block(h_stocks.begin() + i, h_stocks.begin() + i + current_block_size + 1);

        // Transferir o bloco para a GPU
        thrust::device_vector<float> d_stocks = h_stocks_block;

        if (d_stocks.size() > 1) {
            thrust::device_vector<float> d_daily_gain(d_stocks.size() - 1);
            thrust::transform(d_stocks.begin() + 1, d_stocks.end(), d_stocks.begin(), d_daily_gain.begin(), daily_gain_functor());

            // Contar quantos dias tiveram aumento no preço nesse bloco
            int num_positive_gains = thrust::count_if(d_daily_gain.begin(), d_daily_gain.end(), is_positive_gain());
            total_positive_gains += num_positive_gains;
        }
    }

    // Exibir o resultado total
    std::cout << "Número total de dias com aumento no preço: " << total_positive_gains << std::endl;

    return 0;
}

Overwriting contasubida.cu


In [20]:
!nvcc -arch=sm_75 -std=c++14 contasubida.cu -o contasubida

nvcc fatal   : Cannot find compiler 'cl.exe' in PATH


In [21]:
!./contasubida

'.' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


In [22]:
%%writefile aumentomedio.cu
#include <iostream>
#include <fstream>
#include <sstream>
#include <string>
#include <vector>
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/transform.h>
#include <thrust/functional.h>
#include <thrust/count.h>
#include <thrust/replace.h>
#include <thrust/reduce.h>

// Função para ler os preços de ações do arquivo
void read_stock_data(const std::string& file_name, thrust::host_vector<float>& stocks) {
    std::ifstream file(file_name);
    std::string line;

    while (std::getline(file, line)) {
        float price = std::stof(line); // Converte a string para float
        stocks.push_back(price);       // Armazena no vetor
    }
}

// Functor para calcular a diferença entre dias consecutivos
struct daily_gain_functor {
    __host__ __device__
    float operator()(const float& current, const float& previous) const {
        return current - previous;  // Calcula ganho diário
    }
};

// Functor para identificar dias de aumento no preço (diferença positiva)
struct is_positive_gain {
    __host__ __device__
    bool operator()(const float& x) const {
        return x > 0.0f;  // Retorna verdadeiro se o ganho for positivo
    }
};

// Functor para substituir ganhos negativos por zero
struct is_negative_gain {
    __host__ __device__
    bool operator()(const float& x) const {
        return x < 0.0f;  // Retorna verdadeiro se o ganho for negativo
    }
};

int main() {
    thrust::host_vector<float> h_stocks;
    read_stock_data("google.txt", h_stocks);

    size_t total_size = h_stocks.size();
    const size_t block_size = 100000; // Tamanho do bloco a ser processado (ajustável)

    int total_positive_gains = 0;
    float total_sum_positive_gains = 0.0f;

    for (size_t i = 0; i < total_size - 1; i += block_size) {
        size_t current_block_size = std::min(block_size, total_size - i - 1);

        // Bloco atual de dados
        thrust::host_vector<float> h_stocks_block(h_stocks.begin() + i, h_stocks.begin() + i + current_block_size + 1);

        // Transferir o bloco para a GPU
        thrust::device_vector<float> d_stocks = h_stocks_block;

        if (d_stocks.size() > 1) {
            // Calcular as diferenças diárias
            thrust::device_vector<float> d_daily_gain(d_stocks.size() - 1);
            thrust::transform(d_stocks.begin() + 1, d_stocks.end(), d_stocks.begin(), d_daily_gain.begin(), daily_gain_functor());

            // Contar quantos dias tiveram aumento no preço nesse bloco
            int num_positive_gains = thrust::count_if(d_daily_gain.begin(), d_daily_gain.end(), is_positive_gain());
            total_positive_gains += num_positive_gains;

            // Substituir valores negativos por zero
            thrust::replace_if(d_daily_gain.begin(), d_daily_gain.end(), is_negative_gain(), 0.0f);

            // Somar os valores restantes (apenas os positivos)
            float sum_positive_gains = thrust::reduce(d_daily_gain.begin(), d_daily_gain.end(), 0.0f);
            total_sum_positive_gains += sum_positive_gains;
        }
    }

    // Calcular a média dos ganhos positivos
    float average_gain = (total_positive_gains > 0) ? total_sum_positive_gains / total_positive_gains : 0.0f;

    // Exibir os resultados
    std::cout << "Número total de dias com aumento no preço: " << total_positive_gains << std::endl;
    std::cout << "Aumento médio nos dias com ganho: " << average_gain << std::endl;

    return 0;
}

Overwriting aumentomedio.cu


In [23]:
!nvcc -arch=sm_75 -std=c++14 aumentomedio.cu -o aumentomedio

nvcc fatal   : Cannot find compiler 'cl.exe' in PATH


In [24]:
!./aumentomedio

'.' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.
